In [1]:
from scipy.io import arff
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score
from keras import Sequential
from keras.layers import Dense
from keras.optimizers import Adam

Using TensorFlow backend.


In [6]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:56% !important; }</style>"))
pd.options.mode.chained_assignment = None

In [ ]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Pokaż/ukryj kod"></form>''')

# Przygotowanie danych

In [2]:
data = arff.loadarff('rethinpathy.arff')
df = pd.DataFrame(data[0])
df[df.columns[0:8]] = df[df.columns[0:8]].astype(int)
df[df.columns[18:20]] = df[df.columns[18:20]].astype(int)
col_names = ['Quality', 'Pre-screening', 'MA result1', 'MA result2', 'MA result3', 'MA result4', 'MA result5', 'MA result6', 'Exudates result1', 'Exudates result2', 'Exudates result3', 'Exudates result4', 'Exudates result5', 'Exudates result6', 'Exudates result7',
            'Exudates result 8', 'Macula - Optic disc dist', 'Optic disc diameter', 'AM/FM classification', 'Class']
df.columns = col_names
df.head()

,Quality,Pre-screening,MA result1,MA result2,MA result3,MA result4,MA result5,MA result6,Exudates result1,Exudates result2,Exudates result3,Exudates result4,Exudates result5,Exudates result6,Exudates result7,Exudates result 8,Macula - Optic disc dist,Optic disc diameter,AM/FM classification,Class
0,1,1,22,22,22,19,18,14,49.895756,17.775994,5.270920,0.771761,0.018632,0.006864,0.003923,0.003923,0.486903,0.100025,1,0
1,1,1,24,24,22,18,16,13,57.709936,23.799994,3.325423,0.234185,0.003903,0.003903,0.003903,0.003903,0.520908,0.144414,0,0
2,1,1,62,60,59,54,47,33,55.831441,27.993933,12.687485,4.852282,1.393889,0.373252,0.041817,0.007744,0.530904,0.128548,0,1
3,1,1,55,53,53,50,43,31,40.467228,18.445954,9.118901,3.079428,0.840261,0.272434,0.007653,0.001531,0.483284,0.114790,0,0
4,1,1,44,44,44,41,39,27,18.026254,8.570709,0.410381,0.000000,0.000000,0.000000,0.000000,0.000000,0.475935,0.123572,0,1


In [8]:
df[['Quality', 'Pre-screening', 'MA result1', 'Exudates result1',  'Macula - Optic disc dist', 'Optic disc diameter', 'AM/FM classification', 'Class']].head()

,Quality,Pre-screening,MA result1,Exudates result1,Macula - Optic disc dist,Optic disc diameter,AM/FM classification,Class
0,1,1,22,49.895756,0.486903,0.100025,1,0
1,1,1,24,57.709936,0.520908,0.144414,0,0
2,1,1,62,55.831441,0.530904,0.128548,0,1
3,1,1,55,40.467228,0.483284,0.114790,0,0
4,1,1,44,18.026254,0.475935,0.123572,0,1


### Współczynnik pearsona dla całych danych

In [ ]:
dfpc = df.corr(method='pearson')
plt.subplots(figsize=(20,15))
sb.heatmap(dfpc, 
            xticklabels=dfpc.columns,
            yticklabels=dfpc.columns,
            cmap='RdBu_r',
            annot=True,
            linewidth=0.5)

In [ ]:
tetniaki = df.iloc[:, [2, 3, 4, 5, 6, 7, 19]]
wysieki = df.iloc[:, [8, 9, 10, 11, 12, 13, 14, 15, 19]]

# Tętniaki

In [ ]:
tetniaki_zdrowi = tetniaki[tetniaki['Class'] == 0]
tetniaki_chorzy = tetniaki[tetniaki['Class'] == 1]

In [ ]:
tetniaki_zdrowi['mean_ma_count'] = tetniaki_zdrowi.iloc[:,0:7].mean(axis=1).round(2)
print('Średnia liczba tętniaków u pacjentów bez rozpoznaniej retinopatii:', tetniaki_zdrowi['mean_ma_count'].mean())

In [ ]:
tetniaki_chorzy['mean_ma_count'] = tetniaki_chorzy.iloc[:,0:7].mean(axis=1).round(2)
print('Średnia liczba tętniaków u pacjentów z rozpoznaną retinopatią:',tetniaki_chorzy['mean_ma_count'].mean())

### Wykres współczynnika Pearsona dla badania tętniaków

In [ ]:
PCoefTetniaki = tetniaki.corr(method='pearson')
sb.heatmap(PCoefTetniaki, 
            xticklabels=PCoefTetniaki.columns,
            yticklabels=PCoefTetniaki.columns,
            cmap='RdBu_r',
            annot=True,
            linewidth=0.5)

# Wysieki

In [ ]:
wysieki_zdrowi = wysieki[wysieki['Class'] == 0]
wysieki_chorzy = wysieki[wysieki['Class'] == 1]

In [ ]:
wysieki_zdrowi['mean_ma_count'] = wysieki_zdrowi.iloc[:,0:9].mean(axis=1).round(2)
print('Średnia liczba wysieków u pacjentów bez rozpoznaniej retinopatii:', wysieki_zdrowi['mean_ma_count'].mean())

In [ ]:
wysieki_chorzy['mean_ma_count'] = wysieki_chorzy.iloc[:,0:9].mean(axis=1).round(2)
print('Średnia liczba tętniaków u pacjentów z rozpoznaną retinopatią:', wysieki_chorzy['mean_ma_count'].mean())

### Wykres współczynnika Pearsona dla badania wysieków

In [ ]:
PCoefWysieki = wysieki.corr(method='pearson')
sb.heatmap(PCoefWysieki, 
            xticklabels=PCoefWysieki.columns,
            yticklabels=PCoefWysieki.columns,
            cmap='RdBu_r',
            annot=True,
            linewidth=0.5)

# Współczynnik pearsona między odległością od środka plamki żółtej do środka tarczy nerwu wzrokowego a średnicą tarczy nerwu wzrokowego

In [ ]:
odl = df.iloc[:, [16, 17]]
odl = odl.corr(method='pearson')
odl

# Klasyfikacja danych z uwzględnieniem danych ze współczynnikiem Pearsona wyższym niż 0,2

In [ ]:
#klasyfikator dla wsp pearsona >= 0.2
DataHighpeardf = df.iloc[:, [2, 3, 4, 5]]
target = df.iloc[:, 19]

data_train, data_test, target_train, target_test = train_test_split(DataHighpeardf,target, test_size = 0.10, random_state = 10)

svc_model = LinearSVC(random_state=0, max_iter = 10000000)

pred = svc_model.fit(data_train, target_train).predict(data_test)

print("LinearSVC accuracy: ", accuracy_score(target_test, pred, normalize = True))

In [ ]:
#neural network
adam = Adam(lr=0.01)

classifier = Sequential()
classifier.add(Dense(5, activation='sigmoid', kernel_initializer='random_normal', input_dim=4))
classifier.add(Dense(5, activation='sigmoid', kernel_initializer='random_normal'))
classifier.add(Dense(5, activation='sigmoid', kernel_initializer='random_normal'))
classifier.add(Dense(1, activation='sigmoid', kernel_initializer='random_normal'))
classifier.compile(optimizer=adam, loss='binary_crossentropy', metrics=['accuracy'])
classifier.fit(data_train, target_train, epochs=1000, verbose=0)

prednn = classifier.predict(data_test)

In [ ]:
prednn = (prednn > 0.5)
print("Neural Network accuracy: ", accuracy_score(target_test, prednn))

# Klasyfikacja danych z uwzględnieniem danych ze współczynnikiem Pearsona wyższym niż 0,15

In [ ]:
#dla pearsona >= 0.15
DataSemiHighpeardf = df.iloc[:, [2, 3, 4, 5, 13, 14, 15]]
target = df.iloc[:, 19]

data_train, data_test, target_train, target_test = train_test_split(DataSemiHighpeardf,target, test_size = 0.10, random_state = 10)

svc_model = LinearSVC(random_state=0, max_iter = 10000000)

pred = svc_model.fit(data_train, target_train).predict(data_test)

print("LinearSVC acc: ", accuracy_score(target_test, pred, normalize = True))

In [ ]:
#neural network
adam = Adam(lr=0.01)

classifier = Sequential()
classifier.add(Dense(5, activation='sigmoid', kernel_initializer='random_normal', input_dim=7))
classifier.add(Dense(5, activation='sigmoid', kernel_initializer='random_normal'))
classifier.add(Dense(5, activation='sigmoid', kernel_initializer='random_normal'))
classifier.add(Dense(1, activation='sigmoid', kernel_initializer='random_normal'))
classifier.compile(optimizer=adam, loss='binary_crossentropy', metrics=['accuracy'])
classifier.fit(data_train, target_train, epochs=1000, verbose=0)

prednn2 = classifier.predict(data_test)
prednn2 = (prednn2 > 0.5)
print("NN accuracy: ", accuracy_score(target_test, prednn2))

# Dyskretyzacja

In [ ]:
datadisc = arff.loadarff('rethinpathydisc.arff')
dfdisc = pd.DataFrame(data[0])
dfdisc[dfdisc.columns[0:8]] = dfdisc[dfdisc.columns[0:8]]
dfdisc[dfdisc.columns[18:20]] = dfdisc[dfdisc.columns[18:20]]
col_names = ['Quality', 'Pre-screening', 'MA result1', 'MA result2', 'MA result3', 'MA result4', 'MA result5', 'MA result6', 'Exudates result1', 'Exudates result2', 'Exudates result3', 'Exudates result4', 'Exudates result5', 'Exudates result6', 'Exudates result7',
            'Exudates result 8', 'Macula - Optic disc dist', 'Optic disc diameter', 'AM/FM classification', 'Class']
dfdisc.columns = col_names
dfdisc.head()

Przeprowadzenie dyskretyzacji danych za pomocą programu Weka nie przyniosła żadnej różnicy w danych, więc nie zostały przeprowadzone dalsze testy.